# Image Preprocessing

## Setup

In [7]:
import cv2
import os
from PIL import Image
import glob
import numpy as np
import io

In [2]:
!curl -O http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.0M  100 61.0M    0     0  3815k      0  0:00:16  0:00:16 --:--:-- 5284k
bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


## Face and Label Detection

### 1. Using Google Vision API

#### 1.1 Import Google Vision Library

In [3]:
from google.cloud import vision

#### 1.2 Label Detection function

In [14]:
def detect_labels_google(path):
    """Detects labels in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        print(label.description)
    
    labels_list = [label.description for label in labels]
    
    return labels_list

#### 1.3 Face Detection function

In [15]:
def detect_face_google(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = vision.types.Image(content=content)

    return client.face_detection(image=image).face_annotations

#### 1.4 Function to crop the detected faces

In [16]:
def crop_faces_google(image_file, cropped_images_path, faces):
    count = 1
    # open the image
    image = Image.open(image_file)
    
#     print(len(faces))

    # if faces is null, then it means no face was detected in the image
    if not faces:
        print("No face detected in the image.")
        return
    
    # for each detected face in the faces list
    for face in faces:
        # get the coordinates for each vertex
        coordinates = [(vertex.x, vertex.y)
            for vertex in face.bounding_poly.vertices]
        
        # separate the x and y coordinates
        x_coordinates, y_coordinates = [], []
        for vertex in face.bounding_poly.vertices:
            x_coordinates.append(vertex.x)
            y_coordinates.append(vertex.y)
        
        x0, x1, y0, y1 = x_coordinates[0], x_coordinates[2], y_coordinates[0], y_coordinates[2]
        
        # set the coordinates of the box for each face
        box = (x0, y0, x1, y1)
        # crop the image using coordinates of the box
        cropped_image = image.crop(box)
        
        # extract image name from filename
        image_name = (image_file.split("/")[-1])[:-4]
        # save the cropped image
        cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
        count+=1

#### 1.5 Function to resize the cropped faces

In [17]:
def resize_faces_google(cropped_images_path, scaled_images_path, size):
    count = 1
    # for each image in the cropped images path
    for file in glob.glob(cropped_images_path+"*.jpg"):
        # read the image
        image = cv2.imread(file)
        # get the height and width of the image
        height, width = image.shape[:2]
        
        # get the height and weight ratios
        height_ratio, width_ratio = float(size/height), float(size/width)

        # resize the image making sure that the original ratio is maintained
        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
        # extract image name from full file name
        image_name = (file.split("/")[-1])
        # save the scaled image
        cv2.imwrite(scaled_images_path + image_name, resized)

#### 1.6 Apply preprocessing to the dataset using the functions above

In [19]:
image_categories = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/"]]

# for each image_category in the image_categories list
for image_category in image_categories:
    # for each image in the original dataset directory
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        with open(image_file, 'rb') as image:
            # detect faces in the image
            faces = detect_face_google(image)
            
            # Reset file pointer, so we can read the file again
            image.seek(0)
            # crop detected faces and save in "Faces" directory
            crop_faces_google(image_file, image_category[1], faces)
#         break

    # resize the cropped faces stored in the "Faces" directory and save in "Scaled" directory
    resize_faces_google(image_category[1], image_category[2], 64)
#     break

emotiw/train/Positive/pos_1.jpg


---

### 2. Using OpenCV

#### 2.1 Method 1: Using DNN

##### 2.1.1 Load the serialized model from disk

In [20]:
# load our serialized model from disk
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt',
                               'res10_300x300_ssd_iter_140000.caffemodel'
                               )

##### 2.1.2 Function to detect and crop faces using DNN

In [2]:
def extract_faces_cv_dnn(image_file, cropped_images_path):
    # load the input image and construct an input blob for the image
    # by resizing to a fixed 300x300 pixels and then normalizing it
    image = cv2.imread(image_file)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300,
                                 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    faces = net.forward()
    count = 1
    image = Image.open(image_file)
    
    if faces is None:
        print("No face detected in the image.")
        return

    # loop over the faces
    for i in range(faces.shape[2]):

        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = faces[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.5:

            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            cropped_image = image.crop(box)

            image_name = (image_file.split("/")[-1])[:-4]
            cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
            count+=1

---

#### 2.2 Method 2: Using Haar Cascade

##### 2.2.1 Load the serialized model from disk

In [3]:
# Create the haar cascade
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

##### 2.2.2 Function to detect and crop faces using Haar Cascade

In [4]:
def extract_faces_cv(image_file, cropped_images_path):
    # Read the image
    image = cv2.imread(image_file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    count = 0
    image = Image.open(image_file)
    
    if faces is None:
        print("No face detected in the image.")
        return

    # loop over the faces
    for (x, y, w, h) in faces:
        # crop the face in the image
        cropped_image = image.crop((x, y, x+w, y+h))
        image_name = (image_file.split("/")[-1])[:-4]
        cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
        count+=1
    return count

#### 2.3 Function to resize the cropped faces

In [22]:
def resize_faces_cv(cropped_images_path, scaled_images_path, size):
    count = 1
    # for each image in the cropped images path
    for file in glob.glob(cropped_images_path+"*.jpg"):
        # read the image
        image = cv2.imread(file)
        # get the height and width of the image
        height, width = image.shape[:2]
        
        # get the height and weight ratios
        height_ratio, width_ratio = float(size/height), float(size/width)

        # resize the image making sure that the original ratio is maintained
        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
        # extract image name from full file name
        image_name = (file.split("/")[-1])
        # save the scaled image
        cv2.imwrite(scaled_images_path + image_name, resized)

#### 2.4 Apply preprocessing to the dataset using the functions above

In [24]:
image_categories = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/"]]

# for each image_category in the image_categories list
for image_category in image_categories:
    # for each image in the original dataset directory
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        # detect and crop faces in the image
        extract_faces_cv(image_file, image_category[1])
        break
    # resize the cropped faces stored in the "Faces" directory and save in "Scaled" directory
    resize_faces_cv(image_category[1], image_category[2], 64)

emotiw/train/Positive/pos_1.jpg
emotiw/val/Positive/pos_1.jpg
emotiw/val/Neutral/neu_1.jpg
emotiw/val/Negative/neg_1.jpg
emotiw/train/Neutral/neu_1.jpg
emotiw/train/Negative/neg_1.jpg


---

### 3. Using MTCNN

#### 3.1 Import MTCNN

In [1]:
from mtcnn.mtcnn import MTCNN

#### 3.2 Initialize the model

In [2]:
mtcnn = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


#### 3.3 Function to detect and crop faces using MTCNN

In [3]:
def extract_faces_mtcnn(image_file, cropped_images_path):
    # Read the image
    image = cv2.imread(image_file)

    faces = mtcnn.detect_faces(image)
    print(faces)

    count = 0
    image = Image.open(image_file)
    
    if faces is None:
        print("No face detected in the image.")
        return

    # loop over the faces
    for face in faces:
        bounding_box = face['box']
        # crop the face in the image
        cropped_image = image.crop((bounding_box[0], bounding_box[1], bounding_box[0]+bounding_box[2], bounding_box[1]+bounding_box[3]))
        image_name = (image_file.split("/")[-1])[:-4]
        cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
        count+=1
    return count

---

### 4. Using Dlib

#### 4.1 Import Dlib

In [6]:
import dlib

#### 4.2 Function to detect and crop faces

In [27]:
# dlib hog + svm based face detector
detector = dlib.get_frontal_face_detector()

def extract_faces(image_file, cropped_images_path):
    # load input image
    image = cv2.imread(image_file)
    count = 1
    # get the image height and width
    image_height, image_width = image.shape[:2]
    
    if image is None:
        print("Could not read input image")
        exit()
    
    # apply face detection
    faces = detector(image, 1)
    
    # loop over detected faces
    for face in faces:
        # crop the image
        cropped_image = image[max(0, face.top()): min(face.bottom(), image_height),
                    max(0, face.left()): min(face.right(), image_width)]
        # extract image name from filename
        image_name = (image_file.split("/")[-1])[:-4]
        # save the cropped image
        cv2.imwrite(cropped_images_path + image_name + "_face_" + str(count) + ".jpg", cropped_image)
        count+=1

#### 4.3 Function to resize the cropped faces

In [28]:
def resize_faces(cropped_images_path, scaled_images_path, size):
    count = 1
    # for each image in the cropped images path
    for file in glob.glob(cropped_images_path+"*.jpg"):
        # read the image
        image = cv2.imread(file)
        # get the height and width of the image
        height, width = image.shape[:2]
        
        # get the height and weight ratios
        height_ratio, width_ratio = float(size/height), float(size/width)
        
        # resize the image making sure that the original ratio is maintained
        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
        # extract image name from full file name
        image_name = (file.split("/")[-1])
        # save the scaled image
        cv2.imwrite(scaled_images_path + image_name, resized)

#### 4.4 Dlib Align Class

In [8]:
# Copyright 2015-2016 Carnegie Mellon University
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Module for dlib-based alignment."""

import cv2
import dlib
import numpy as np


TEMPLATE = np.float32([
    (0.0792396913815, 0.339223741112), (0.0829219487236, 0.456955367943),
    (0.0967927109165, 0.575648016728), (0.122141515615, 0.691921601066),
    (0.168687863544, 0.800341263616), (0.239789390707, 0.895732504778),
    (0.325662452515, 0.977068762493), (0.422318282013, 1.04329000149),
    (0.531777802068, 1.06080371126), (0.641296298053, 1.03981924107),
    (0.738105872266, 0.972268833998), (0.824444363295, 0.889624082279),
    (0.894792677532, 0.792494155836), (0.939395486253, 0.681546643421),
    (0.96111933829, 0.562238253072), (0.970579841181, 0.441758925744),
    (0.971193274221, 0.322118743967), (0.163846223133, 0.249151738053),
    (0.21780354657, 0.204255863861), (0.291299351124, 0.192367318323),
    (0.367460241458, 0.203582210627), (0.4392945113, 0.233135599851),
    (0.586445962425, 0.228141644834), (0.660152671635, 0.195923841854),
    (0.737466449096, 0.182360984545), (0.813236546239, 0.192828009114),
    (0.8707571886, 0.235293377042), (0.51534533827, 0.31863546193),
    (0.516221448289, 0.396200446263), (0.517118861835, 0.473797687758),
    (0.51816430343, 0.553157797772), (0.433701156035, 0.604054457668),
    (0.475501237769, 0.62076344024), (0.520712933176, 0.634268222208),
    (0.565874114041, 0.618796581487), (0.607054002672, 0.60157671656),
    (0.252418718401, 0.331052263829), (0.298663015648, 0.302646354002),
    (0.355749724218, 0.303020650651), (0.403718978315, 0.33867711083),
    (0.352507175597, 0.349987615384), (0.296791759886, 0.350478978225),
    (0.631326076346, 0.334136672344), (0.679073381078, 0.29645404267),
    (0.73597236153, 0.294721285802), (0.782865376271, 0.321305281656),
    (0.740312274764, 0.341849376713), (0.68499850091, 0.343734332172),
    (0.353167761422, 0.746189164237), (0.414587777921, 0.719053835073),
    (0.477677654595, 0.706835892494), (0.522732900812, 0.717092275768),
    (0.569832064287, 0.705414478982), (0.635195811927, 0.71565572516),
    (0.69951672331, 0.739419187253), (0.639447159575, 0.805236879972),
    (0.576410514055, 0.835436670169), (0.525398405766, 0.841706377792),
    (0.47641545769, 0.837505914975), (0.41379548902, 0.810045601727),
    (0.380084785646, 0.749979603086), (0.477955996282, 0.74513234612),
    (0.523389793327, 0.748924302636), (0.571057789237, 0.74332894691),
    (0.672409137852, 0.744177032192), (0.572539621444, 0.776609286626),
    (0.5240106503, 0.783370783245), (0.477561227414, 0.778476346951)])

TPL_MIN, TPL_MAX = np.min(TEMPLATE, axis=0), np.max(TEMPLATE, axis=0)
MINMAX_TEMPLATE = (TEMPLATE - TPL_MIN) / (TPL_MAX - TPL_MIN)


class AlignDlib:
    """
    Use `dlib's landmark estimation <http://blog.dlib.net/2014/08/real-time-face-pose-estimation.html>`_ to align faces.

    The alignment preprocess faces for input into a neural network.
    Faces are resized to the same size (such as 96x96) and transformed
    to make landmarks (such as the eyes and nose) appear at the same
    location on every image.

    Normalized landmarks:

    .. image:: ../images/dlib-landmark-mean.png
    """

    #: Landmark indices.
    INNER_EYES_AND_BOTTOM_LIP = [39, 42, 57]
    OUTER_EYES_AND_NOSE = [36, 45, 33]

    def __init__(self, facePredictor):
        """
        Instantiate an 'AlignDlib' object.

        :param facePredictor: The path to dlib's
        :type facePredictor: str
        """
        assert facePredictor is not None

        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(facePredictor)

    def getAllFaceBoundingBoxes(self, rgbImg):
        """
        Find all face bounding boxes in an image.

        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :return: All face bounding boxes in an image.
        :rtype: dlib.rectangles
        """
        assert rgbImg is not None

        try:
            return self.detector(rgbImg, 1)
        except Exception as e:
            print("Warning: {}".format(e))
            # In rare cases, exceptions are thrown.
            return []

    def getLargestFaceBoundingBox(self, rgbImg, skipMulti=False):
        """
        Find the largest face bounding box in an image.

        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :param skipMulti: Skip image if more than one face detected.
        :type skipMulti: bool
        :return: The largest face bounding box in an image, or None.
        :rtype: dlib.rectangle
        """
        assert rgbImg is not None

        faces = self.getAllFaceBoundingBoxes(rgbImg)
        if (not skipMulti and len(faces) > 0) or len(faces) == 1:
            return max(faces, key=lambda rect: rect.width() * rect.height())
        else:
            return None

    def findLandmarks(self, rgbImg, bb):
        """
        Find the landmarks of a face.

        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :param bb: Bounding box around the face to find landmarks for.
        :type bb: dlib.rectangle
        :return: Detected landmark locations.
        :rtype: list of (x,y) tuples
        """
        assert rgbImg is not None
        assert bb is not None

        points = self.predictor(rgbImg, bb)
        return list(map(lambda p: (p.x, p.y), points.parts()))

    def align(self, imgDim, rgbImg, bb=None,
              landmarks=None, landmarkIndices=INNER_EYES_AND_BOTTOM_LIP,
              skipMulti=False):
        r"""align(imgDim, rgbImg, bb=None, landmarks=None, landmarkIndices=INNER_EYES_AND_BOTTOM_LIP)

        Transform and align a face in an image.

        :param imgDim: The edge length in pixels of the square the image is resized to.
        :type imgDim: int
        :param rgbImg: RGB image to process. Shape: (height, width, 3)
        :type rgbImg: numpy.ndarray
        :param bb: Bounding box around the face to align. \
                   Defaults to the largest face.
        :type bb: dlib.rectangle
        :param landmarks: Detected landmark locations. \
                          Landmarks found on `bb` if not provided.
        :type landmarks: list of (x,y) tuples
        :param landmarkIndices: The indices to transform to.
        :type landmarkIndices: list of ints
        :param skipMulti: Skip image if more than one face detected.
        :type skipMulti: bool
        :return: The aligned RGB image. Shape: (imgDim, imgDim, 3)
        :rtype: numpy.ndarray
        """
        assert imgDim is not None
        assert rgbImg is not None
        assert landmarkIndices is not None

        if bb is None:
            bb = self.getLargestFaceBoundingBox(rgbImg, skipMulti)
            if bb is None:
                return

        if landmarks is None:
            landmarks = self.findLandmarks(rgbImg, bb)

        npLandmarks = np.float32(landmarks)
        npLandmarkIndices = np.array(landmarkIndices)

        H = cv2.getAffineTransform(npLandmarks[npLandmarkIndices],
                                   imgDim * MINMAX_TEMPLATE[npLandmarkIndices])
        thumbnail = cv2.warpAffine(rgbImg, H, (imgDim, imgDim))

        return thumbnail

#### 4.5 Function to align the faces

In [12]:
align_dlib = AlignDlib('shape_predictor_68_face_landmarks.dat')

def align_faces(scaled_images_path, aligned_images_path):
    count = 1
    # for each image in the scaled images directory
    for file in glob.glob(scaled_images_path+"*.jpg"):
        
        print(file)
        
        # read the image
        image = cv2.imread(file)
        
        # initialize the bounding box
        bb = align_dlib.getLargestFaceBoundingBox(image)
        # align the face
        aligned = align_dlib.align(64, image, bb, landmarkIndices=AlignDlib.INNER_EYES_AND_BOTTOM_LIP)
        image_name = (file.split("/")[-1])
        # if aligned
        if aligned is not None:
            # save the image in the aligned images directory
            cv2.imwrite(aligned_images_path + image_name, aligned)
        else:
            # save the image without alignment in the aligned images directory
            cv2.imwrite(aligned_images_path + image_name, image)

#### 4.6 Apply preprocessing to the dataset using the functions above

In [29]:
image_categories = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/", "Aligned/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/", "Aligned/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/", "Aligned/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/", "Aligned/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/", "Aligned/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/", "Aligned/train/Negative/"]]

# for each image_category in the image_categories list
for image_category in image_categories:
    # for each image in the original dataset directory
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        # detect and crop faces in the image
        extract_faces(image_file, image_category[1])
    # resize the cropped faces and save in "Scaled" directory
    resize_faces(image_category[1], image_category[2], 64)
    # align the scaled faces and save in "Aligned" directory
    align_faces(image_category[2], image_category[3])

Scaled/train/Positive/pos_1935_face_4.jpg
Scaled/train/Positive/pos_2761_face_2.jpg
Scaled/train/Positive/pos_2618_face_14.jpg
Scaled/train/Positive/pos_2852_face_3.jpg
Scaled/train/Positive/pos_951_face_2.jpg
Scaled/train/Positive/pos_1514_face_1.jpg
Scaled/train/Positive/pos_673_face_1.jpg
Scaled/train/Positive/pos_2534_face_18.jpg
Scaled/train/Positive/pos_3062_face_2.jpg
Scaled/train/Positive/pos_3051_face_11.jpg
Scaled/train/Positive/pos_1968_face_22.jpg
Scaled/train/Positive/pos_3361_face_4.jpg
Scaled/train/Positive/pos_100_face_3.jpg
Scaled/train/Positive/pos_2311_face_6.jpg
Scaled/train/Positive/pos_2003_face_2.jpg
Scaled/train/Positive/pos_1267_face_3.jpg
Scaled/train/Positive/pos_1276_face_1.jpg
Scaled/train/Positive/pos_111_face_1.jpg
Scaled/train/Positive/pos_3700_face_2.jpg
Scaled/train/Positive/pos_2992_face_2.jpg
Scaled/train/Positive/pos_405_face_9.jpg
Scaled/train/Positive/pos_2506_face_8.jpg
Scaled/train/Positive/pos_1778_face_3.jpg
Scaled/train/Positive/pos_3822_face

Scaled/train/Positive/pos_1696_face_6.jpg
Scaled/train/Positive/pos_1584_face_2.jpg
Scaled/train/Positive/pos_1628_face_1.jpg
Scaled/train/Positive/pos_3963_face_1.jpg
Scaled/train/Positive/pos_1687_face_4.jpg
Scaled/train/Positive/pos_2241_face_4.jpg
Scaled/train/Positive/pos_1337_face_1.jpg
Scaled/train/Positive/pos_303_face_12.jpg
Scaled/train/Positive/pos_1987_face_10.jpg
Scaled/train/Positive/pos_1326_face_3.jpg
Scaled/train/Positive/pos_2158_face_8.jpg
Scaled/train/Positive/pos_2142_face_2.jpg
Scaled/train/Positive/pos_2250_face_6.jpg
Scaled/train/Positive/pos_2922_face_10.jpg
Scaled/train/Positive/pos_71_face_4.jpg
Scaled/train/Positive/pos_3284_face_9.jpg
Scaled/train/Positive/pos_181_face_2.jpg
Scaled/train/Positive/pos_2082_face_3.jpg
Scaled/train/Positive/pos_1249_face_7.jpg
Scaled/train/Positive/pos_282_face_4.jpg
Scaled/train/Positive/pos_2093_face_1.jpg
Scaled/train/Positive/pos_3123_face_2.jpg
Scaled/train/Positive/pos_2523_face_4.jpg
Scaled/train/Positive/pos_420_face_5

Scaled/train/Positive/pos_1753_face_2.jpg
Scaled/train/Positive/pos_798_face_1.jpg
Scaled/train/Positive/pos_1441_face_6.jpg
Scaled/train/Positive/pos_2537_face_3.jpg
Scaled/train/Positive/pos_434_face_2.jpg
Scaled/train/Positive/pos_1871_face_1.jpg
Scaled/train/Positive/pos_848_face_1.jpg
Scaled/train/Positive/pos_1883_face_2.jpg
Scaled/train/Positive/pos_1417_face_8.jpg
Scaled/train/Positive/pos_2669_face_3.jpg
Scaled/train/Positive/pos_3464_face_1.jpg
Scaled/train/Positive/pos_2678_face_1.jpg
Scaled/train/Positive/pos_859_face_3.jpg
Scaled/train/Positive/pos_1112_face_2.jpg
Scaled/train/Positive/pos_275_face_2.jpg
Scaled/train/Positive/pos_3278_face_7.jpg
Scaled/train/Positive/pos_2376_face_3.jpg
Scaled/train/Positive/pos_2316_face_28.jpg
Scaled/train/Positive/pos_2567_face_58.jpg
Scaled/train/Positive/pos_2367_face_1.jpg
Scaled/train/Positive/pos_1211_face_4.jpg
Scaled/train/Positive/pos_86_face_2.jpg
Scaled/train/Positive/pos_3317_face_3.jpg
Scaled/train/Positive/pos_3306_face_1.j

Scaled/train/Positive/pos_2802_face_10.jpg
Scaled/train/Positive/pos_1384_face_4.jpg
Scaled/train/Positive/pos_3342_face_2.jpg
Scaled/train/Positive/pos_2742_face_4.jpg
Scaled/train/Positive/pos_1916_face_2.jpg
Scaled/train/Positive/pos_553_face_1.jpg
Scaled/train/Positive/pos_641_face_5.jpg
Scaled/train/Positive/pos_963_face_6.jpg
Scaled/train/Positive/pos_1634_face_1.jpg
Scaled/train/Positive/pos_1625_face_3.jpg
Scaled/train/Positive/pos_972_face_4.jpg
Scaled/train/Positive/pos_1537_face_7.jpg
Scaled/train/Positive/pos_2871_face_5.jpg
Scaled/train/Positive/pos_2441_face_2.jpg
Scaled/train/Positive/pos_542_face_3.jpg
Scaled/train/Positive/pos_790_face_6.jpg
Scaled/train/Positive/pos_482_face_2.jpg
Scaled/train/Positive/pos_1449_face_1.jpg
Scaled/train/Positive/pos_1458_face_3.jpg
Scaled/train/Positive/pos_781_face_4.jpg
Scaled/train/Positive/pos_44_face_11.jpg
Scaled/train/Positive/pos_2590_face_1.jpg
Scaled/train/Positive/pos_947_face_17.jpg
Scaled/train/Positive/pos_3420_face_2.jpg


Scaled/train/Positive/pos_1462_face_24.jpg
Scaled/train/Positive/pos_2725_face_1.jpg
Scaled/train/Positive/pos_590_face_9.jpg
Scaled/train/Positive/pos_2437_face_5.jpg
Scaled/train/Positive/pos_1971_face_7.jpg
Scaled/train/Positive/pos_534_face_4.jpg
Scaled/train/Positive/pos_3539_face_1.jpg
Scaled/train/Positive/pos_2734_face_3.jpg
Scaled/train/Positive/pos_3528_face_3.jpg
Scaled/train/Positive/pos_637_face_2.jpg
Scaled/train/Positive/pos_915_face_1.jpg
Scaled/train/Positive/pos_1550_face_2.jpg
Scaled/train/Positive/pos_689_face_5.jpg
Scaled/train/Positive/pos_2284_face_6.jpg
Scaled/train/Positive/pos_3334_face_5.jpg
Scaled/train/Positive/pos_3026_face_1.jpg
Scaled/train/Positive/pos_3037_face_3.jpg
Scaled/train/Positive/pos_3089_face_4.jpg
Scaled/train/Positive/pos_2231_face_9.jpg
Scaled/train/Positive/pos_92_face_4.jpg
Scaled/train/Positive/pos_3011_face_1.jpg
Scaled/train/Positive/pos_1078_face_2.jpg
Scaled/train/Positive/pos_2206_face_9.jpg
Scaled/train/Positive/pos_2984_face_14.j

...

Scaled/train/Negative/neg_1564_face_6.jpg
Scaled/train/Negative/neg_1676_face_2.jpg
Scaled/train/Negative/neg_1954_face_1.jpg
Scaled/train/Negative/neg_2403_face_1.jpg
Scaled/train/Negative/neg_1945_face_3.jpg
Scaled/train/Negative/neg_2687_face_10.jpg
Scaled/train/Negative/neg_1575_face_4.jpg
Scaled/train/Negative/neg_180_face_2.jpg
Scaled/train/Negative/neg_1758_face_1.jpg
Scaled/train/Negative/neg_733_face_1.jpg
Scaled/train/Negative/neg_2032_face_2.jpg
Scaled/train/Negative/neg_421_face_5.jpg
Scaled/train/Negative/neg_1144_face_7.jpg
Scaled/train/Negative/neg_1256_face_3.jpg
Scaled/train/Negative/neg_1247_face_1.jpg
Scaled/train/Negative/neg_1155_face_5.jpg
Scaled/train/Negative/neg_430_face_7.jpg
Scaled/train/Negative/neg_722_face_3.jpg
Scaled/train/Negative/neg_1328_face_5.jpg
Scaled/train/Negative/neg_2442_face_6.jpg
Scaled/train/Negative/neg_343_face_5.jpg
Scaled/train/Negative/neg_1534_face_3.jpg
Scaled/train/Negative/neg_1525_face_1.jpg
Scaled/train/Negative/neg_1689_face_2.j

Scaled/train/Negative/neg_1971_face_11.jpg
Scaled/train/Negative/neg_1959_face_3.jpg
Scaled/train/Negative/neg_304_face_8.jpg
Scaled/train/Negative/neg_1375_face_2.jpg
Scaled/train/Negative/neg_1067_face_6.jpg
Scaled/train/Negative/neg_2111_face_3.jpg
Scaled/train/Negative/neg_2100_face_1.jpg
Scaled/train/Negative/neg_601_face_2.jpg
Scaled/train/Negative/neg_923_face_1.jpg
Scaled/train/Negative/neg_1342_face_2.jpg
Scaled/train/Negative/neg_535_face_4.jpg
Scaled/train/Negative/neg_2137_face_1.jpg
Scaled/train/Negative/neg_1041_face_4.jpg
Scaled/train/Negative/neg_914_face_1.jpg
Scaled/train/Negative/neg_2439_face_3.jpg
Scaled/train/Negative/neg_329_face_2.jpg
Scaled/train/Negative/neg_397_face_5.jpg
Scaled/train/Negative/neg_1420_face_2.jpg
Scaled/train/Negative/neg_2655_face_1.jpg
Scaled/train/Negative/neg_1293_face_1.jpg
Scaled/train/Negative/neg_1282_face_3.jpg
Scaled/train/Negative/neg_2014_face_1.jpg
Scaled/train/Negative/neg_715_face_2.jpg
Scaled/train/Negative/neg_837_face_1.jpg


Scaled/train/Negative/neg_22_face_2.jpg
Scaled/train/Negative/neg_274_face_1.jpg
Scaled/train/Negative/neg_1700_face_1.jpg
Scaled/train/Negative/neg_849_face_2.jpg
Scaled/train/Negative/neg_2070_face_8.jpg
Scaled/train/Negative/neg_853_face_8.jpg
Scaled/train/Negative/neg_1073_face_3.jpg
Scaled/train/Negative/neg_604_face_5.jpg
Scaled/train/Negative/neg_2105_face_6.jpg
Scaled/train/Negative/neg_998_face_1.jpg
Scaled/train/Negative/neg_2217_face_2.jpg
Scaled/train/Negative/neg_516_face_1.jpg
Scaled/train/Negative/neg_1_face_1.jpg
Scaled/train/Negative/neg_507_face_3.jpg
Scaled/train/Negative/neg_989_face_3.jpg
Scaled/train/Negative/neg_1062_face_1.jpg
Scaled/train/Negative/neg_1901_face_2.jpg
Scaled/train/Negative/neg_1623_face_1.jpg
Scaled/train/Negative/neg_1632_face_3.jpg
Scaled/train/Negative/neg_357_face_1.jpg
Scaled/train/Negative/neg_1090_face_2.jpg
Scaled/train/Negative/neg_2153_face_8.jpg
Scaled/train/Negative/neg_424_face_3.jpg
Scaled/train/Negative/neg_1141_face_1.jpg
Scaled/

Scaled/train/Negative/neg_1940_face_5.jpg
Scaled/train/Negative/neg_1570_face_2.jpg
Scaled/train/Negative/neg_995_face_2.jpg
Scaled/train/Negative/neg_844_face_1.jpg
Scaled/train/Negative/neg_2067_face_1.jpg
Scaled/train/Negative/neg_766_face_2.jpg
Scaled/train/Negative/neg_2076_face_3.jpg
Scaled/train/Negative/neg_855_face_3.jpg
Scaled/train/Negative/neg_1880_face_4.jpg
Scaled/train/Negative/neg_2578_face_1.jpg
Scaled/train/Negative/neg_979_face_19.jpg
Scaled/train/Negative/neg_2689_face_4.jpg
Scaled/train/Negative/neg_1453_face_2.jpg
Scaled/train/Negative/neg_1872_face_7.jpg
Scaled/train/Negative/neg_2626_face_1.jpg
Scaled/train/Negative/neg_2534_face_5.jpg
Scaled/train/Negative/neg_785_face_3.jpg
Scaled/train/Negative/neg_2396_face_4.jpg
Scaled/train/Negative/neg_2095_face_2.jpg
Scaled/train/Negative/neg_438_face_2.jpg
Scaled/train/Negative/neg_976_face_3.jpg
Scaled/train/Negative/neg_1331_face_2.jpg
Scaled/train/Negative/neg_1023_face_6.jpg
Scaled/train/Negative/neg_967_face_1.jpg


Scaled/train/Negative/neg_2425_face_1.jpg
Scaled/train/Negative/neg_1493_face_5.jpg
Scaled/train/Negative/neg_1781_face_1.jpg
Scaled/train/Negative/neg_2658_face_1.jpg
Scaled/train/Negative/neg_1132_face_2.jpg
Scaled/train/Negative/neg_745_face_4.jpg
Scaled/train/Negative/neg_2356_face_3.jpg
Scaled/train/Negative/neg_2347_face_1.jpg
Scaled/train/Negative/neg_1231_face_4.jpg
Scaled/train/Negative/neg_2019_face_1.jpg
Scaled/train/Negative/neg_895_face_4.jpg
Scaled/train/Negative/neg_1276_face_8.jpg
Scaled/train/Negative/neg_2008_face_3.jpg
Scaled/train/Negative/neg_2506_face_1.jpg
Scaled/train/Negative/neg_51_face_1.jpg
Scaled/train/Negative/neg_1773_face_2.jpg
Scaled/train/Negative/neg_40_face_3.jpg
Scaled/train/Negative/neg_2517_face_3.jpg
Scaled/train/Negative/neg_1851_face_1.jpg
Scaled/train/Negative/neg_2469_face_5.jpg
Scaled/train/Negative/neg_368_face_6.jpg
Scaled/train/Negative/neg_1980_face_2.jpg
Scaled/train/Negative/neg_379_face_4.jpg
Scaled/train/Negative/neg_2264_face_1.jpg


---

In [ ]:
!